In [1]:
!pip install openai langchain-core langchain-text-splitters PyDrive2 numpy tiktoken beautifulsoup4 lxml

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [3]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import requests
from bs4 import BeautifulSoup

# Аутентификация Google Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Загрузка текста из Google Docs (публичный документ)
doc_url = "https://docs.google.com/document/d/1q4l912Re8zuIfBax4FDS3ZppYmVPzER3Si2wrmznddc"

# Экспорт как HTML
export_url = doc_url + "/export?format=html"
response = requests.get(export_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')
text = soup.get_text(separator="\n", strip=True)

print(f"✅ Загружено {len(text)} символов из документа.")

✅ Загружено 239809 символов из документа.


In [6]:
from langchain_text_splitters import CharacterTextSplitter
import tiktoken

# Функция подсчёта токенов
def count_tokens(text):
    enc = tiktoken.get_encoding("cl100k_base")
    return len(enc.encode(text))

# Сначала пробуем разбить по абзацам
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=500,        # ~500 символов
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_text(text)

# Теперь проверим и при необходимости разобьём слишком длинные чанки по токенам
max_tokens = 8000  # оставим запас до лимита 8191
final_chunks = []

for chunk in chunks:
    if count_tokens(chunk) <= max_tokens:
        final_chunks.append(chunk)
    else:
        # Резерв: разбиваем по символам с шагом ~6000 символов (примерно <8000 токенов)
        # Используем простой слайдинг по символам
        start = 0
        step = 6000
        overlap = 200
        while start < len(chunk):
            sub_chunk = chunk[start:start + step]
            if sub_chunk.strip():
                final_chunks.append(sub_chunk)
            start += step - overlap

chunks = final_chunks

# Убедимся, что все чанки в пределах лимита
for i, c in enumerate(chunks):
    tok = count_tokens(c)
    if tok > 8191:
        print(f"⚠️ Чанк {i} всё ещё слишком длинный: {tok} токенов")

print(f"📄 Разбито на {len(chunks)} чанков. Макс. длина чанка: {max(count_tokens(c) for c in chunks)} токенов.")

📄 Разбито на 42 чанков. Макс. длина чанка: 3021 токенов.


In [7]:
import numpy as np
import pickle
from openai import OpenAI

client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Генерация эмбеддингов
print("Генерация эмбеддингов...")
embeddings = []
for i, chunk in enumerate(chunks):
    emb = get_embedding(chunk)
    embeddings.append(emb)
    if (i + 1) % 20 == 0:
        print(f"Обработано {i + 1} чанков...")

embeddings = np.array(embeddings)

# Сохраняем как pickle: (chunks, embeddings)
vector_db = {"chunks": chunks, "embeddings": embeddings}

# Сохраняем локально
with open("vector_db.pkl", "wb") as f:
    pickle.dump(vector_db, f)

# Загружаем на Google Drive
uploaded = drive.CreateFile({'title': 'vector_db.pkl'})
uploaded.SetContentFile('vector_db.pkl')
uploaded.Upload()
print(f"💾 Векторная база сохранена на Google Drive с ID: {uploaded.get('id')}")

Генерация эмбеддингов...
Обработано 20 чанков...
Обработано 40 чанков...
💾 Векторная база сохранена на Google Drive с ID: 1CHkCJ8Hp1_C_M3C0_cSzD9bNvel19Oc0


In [8]:
# Скачиваем файл с Google Drive (по названию)
file_list = drive.ListFile({'q': "'root' in parents and title='vector_db.pkl'"}).GetList()
if not file_list:
    raise FileNotFoundError("Файл vector_db.pkl не найден на Google Drive")

file_id = file_list[0]['id']
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('vector_db_downloaded.pkl')

# Загружаем
with open("vector_db_downloaded.pkl", "rb") as f:
    loaded_db = pickle.load(f)

chunks_loaded = loaded_db["chunks"]
embeddings_loaded = loaded_db["embeddings"]

print(f"✅ Загружено {len(chunks_loaded)} чанков из Google Drive.")

✅ Загружено 42 чанков из Google Drive.


In [9]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def ask_safety_question(question, top_k=3):
    # Получаем эмбеддинг вопроса
    query_emb = get_embedding(question)

    # Ищем наиболее релевантные чанки
    scores = [cosine_similarity(query_emb, emb) for emb in embeddings_loaded]
    top_indices = np.argsort(scores)[-top_k:][::-1]
    top_chunks = [chunks_loaded[i] for i in top_indices]

    context = "\n\n".join(top_chunks)

    # Генерация ответа через OpenAI (без LangChain!)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": (
                    "Вы — эксперт по промышленной безопасности. "
                    "Отвечайте на вопросы строго на основе предоставленной информации. "
                    "Если информации нет, скажите: 'В документе по безопасности эта информация не содержится.'"
                )
            },
            {
                "role": "user",
                "content": f"Контекст:\n{context}\n\nВопрос: {question}\n\nОтвет:"
            }
        ],
        temperature=0.2,
        max_tokens=300
    )

    return response.choices[0].message.content.strip()

In [10]:
question = "Какие требования предъявляются к обучению персонала на опасных производственных объектах?"

answer = ask_safety_question(question)
print("❓ Вопрос:", question)
print("\n✅ Ответ:")
print(answer)

❓ Вопрос: Какие требования предъявляются к обучению персонала на опасных производственных объектах?

✅ Ответ:
В документе по безопасности содержится информация о требованиях к обучению персонала на опасных производственных объектах. Согласно представленному контексту, эксплуатирующая организация обязана устанавливать порядок контроля обучения и периодической проверки знаний персонала, работающего с ограничителями, указателями и регистраторами, а также документально подтверждать его соблюдение с учетом требований руководства (инструкции) по эксплуатации. Также организация должна организовывать считывание данных с регистратора параметров работы ПС, обработку этих данных, выявление нарушений правил эксплуатации ПС, а при отсутствии указаний о сроках считывания данных проводить такие операции не реже одного раза в шесть месяцев.
